In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medium-articles/medium_articles.csv


In [2]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import csv

In [4]:
df=pd.read_csv('/kaggle/input/medium-articles/medium_articles.csv')
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [5]:
df=df[['title','text']]
df.shape

(192368, 2)

In [6]:
df=df.drop_duplicates()
df.shape

(190827, 2)

In [7]:
df['text']=df['text'].str.replace('\n','<|n|>')

In [8]:
sample_blog=df['text'][10]
print(sample_blog)

Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company. Apply for a job, and recruiters ask about your career path. If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories? Because they reveal character. Superman is a good guy because his planet exploded, his parents sent him to Earth, and the Kents raised him to fight evil. Steve Jobs became a trusted CEO, in part, by bouncing back from failure. My dad thrived as a home builder, but only after terminating a partnership with a prominent colleague whose practices he found unethical.<|n|><|n|>In other words:<|n|><|n|>Your origin story helps people decide whether or not to trust you.<|n|><|n|>How to Craft a Powerful Origin Story: The 5 Essential Elements<|n|><|n|>In strategic messaging and positioning engagements with CEOs, as well as business st

In [10]:
sentences = sent_tokenize(sample_blog)
sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')

sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)

sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words]for words in sentence_tokens]
max_len = max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant')for embedding in sentence_embeddings]

In [11]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph, max_iter=600)

top_sentence = {sentence: scores[index] for index, sentence in enumerate(sentences)}
sentNeeded = round(0.25 * len(sentences)) - 1
top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])

summary = ""
for sent in sentences:
    if sent in top.keys():
        summary += sent + " "
print(summary)

Apply for a job, and recruiters ask about your career path. If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories? Steve Jobs became a trusted CEO, in part, by bouncing back from failure. Very frequently, another person illuminates a new way forward.<|n|><|n|>Can you pinpoint the event that set you on your current path? The sign says, “For anyone who wants to tell a compelling story,” and it’s pointing to a book about screenwriting. Basically, describe what happened next:<|n|><|n|>That night, I read the book from cover to cover. It’s written in the language of feature films (three act structure, etc. I rewrite my pitch and send it to a new batch of investors.<|n|><|n|>4. “Until finally…”<|n|><|n|>What was the turning point — the moment when you really committed to the new path? Pay particular attention to where they lean in.<|n|><|n|>Also, ask others to tell you their origin stories. 


In [12]:
count = 0

def generateSummary(blog):
    global count
    count += 1
    print("Summarising blog", count)
    try:
        sentences = sent_tokenize(blog)
        sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
        sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words]for sentence in sentences_clean]
        w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1008)
        sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words]for words in sentence_tokens]
        max_len = max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant')for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
        for i, row_embedding in enumerate(sentence_embeddings):
            for j, column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph, max_iter=600)
        top_sentence = {sentence: scores[index] for index, sentence in enumerate(sentences)}
        sentNeeded = round(0.25 * len(sentences)) - 1
        top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
        summary = ""
        for sent in sentences:
            if sent in top.keys():
                summary += sent + " "
        return summary
    except Exception as e:
        print("Error:", e)
        return float("NaN")

In [ ]:
import csv
filename = "articlesSet.csv"
fields = ['title', 'summary', 'content']
with open(filename, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    def callback(row):
        summary = generateSummary(row['text'])
        if type(summary) != str:
            return
        rows = [row['title'], summary, row['text']]
        with open(filename, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(rows)
    df.apply(callback, axis=1)

Summarising blog 1
Summarising blog 2
Summarising blog 3
Summarising blog 4
Summarising blog 5
Summarising blog 6
Summarising blog 7
Summarising blog 8
Summarising blog 9
Summarising blog 10
Summarising blog 11
Summarising blog 12
Summarising blog 13
Summarising blog 14
Summarising blog 15
Summarising blog 16
Summarising blog 17
Summarising blog 18
Summarising blog 19
Summarising blog 20
Summarising blog 21
Summarising blog 22
Summarising blog 23


/opt/conda/lib/python3.10/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 24
Summarising blog 25
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 26
Summarising blog 27
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 28
Summarising blog 29
Summarising blog 30
Summarising blog 31
Summarising blog 32
Summarising blog 33
Summarising blog 34
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 35
Summarising blog 36
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 37
Summarising blog 38
Summarising blog 39
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 40
Summarising blog 41
Summ

/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 178
Summarising blog 179
Summarising blog 180
Summarising blog 181
Summarising blog 182
Summarising blog 183
Summarising blog 184
Summarising blog 185
Summarising blog 186
Summarising blog 187
Summarising blog 188
Summarising blog 189
Summarising blog 190
Summarising blog 191
Summarising blog 192
Summarising blog 193
Summarising blog 194
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 195
Summarising blog 196
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 197
Summarising blog 198
Summarising blog 199
Summarising blog 200
Summarising blog 201
Summarising blog 202
Summarising blog 203
Summarising blog 204
Summarising blog 205
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 i

/opt/conda/lib/python3.10/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:496: RuntimeWarning: invalid value encountered in subtract
  err = np.absolute(x - xlast).sum()


Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 1054
Summarising blog 1055
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 1056
Summarising blog 1057
Summarising blog 1058
Summarising blog 1059
Summarising blog 1060
Summarising blog 1061
Summarising blog 1062
Summarising blog 1063
Summarising blog 1064
Summarising blog 1065
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 600 iterations')
Summarising blog 1066
Summarising blog 1067
Summarising blog 1068
Summarising blog 1069
Summarising blog 1070
Summarising blog 1071
Summarising blog 1072
Summarising blog 1073
Summarising blog 1074
Summarising blog 1075
Summarising blog 1076
Summarising blog 1077
Summarising blog 1078
Summarising blog 1079
Summarising blog 1080
Error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within